# Exploring the Citymapper API

> Aim: Explore and understand the output of the Citymapper API.

## Load modules

In [1]:
import os
import requests
import datetime
from dotenv import load_dotenv

import pandas as pd

## Querying the travel time between two stations

There looks to be two endpoints that are suitable for our query of interest:

- [https://api.external.citymapper.com/api/1/traveltimes](http://docs.external.citymapper.com/api/#operation/traveltime)
- [https://api.external.citymapper.com/api/1/directions/transit](http://docs.external.citymapper.com/api/#operation/transitdirections)

Let's test both.

### Traveltimes

Fairly straightforward API, giving us the total transit time. However, does not let us query by the departure time.

In [2]:
# loads our API key and the base URL
load_dotenv()

base_cm_url = os.getenv("CM_API_URL")
headers = {"Citymapper-Partner-Key": os.getenv("CM_API_KEY")}

leyton_coords = "51.55609689,-0.00386474"
harrow_on_the_hill_coords = "51.57891586,-0.33516664"

params = {"start": leyton_coords, "end": harrow_on_the_hill_coords}

In [3]:
cm_response = requests.get(
    url=base_cm_url + "1/traveltimes",
    params=params,
    headers=headers
)

In [4]:
cm_response.json()

{'walk_travel_time_minutes': 378, 'transit_time_minutes': 62}

### Transit

In [5]:
# get the date of next tuesday in ISO 8601 format
next_tues = datetime.datetime(2022, 9, 27, 8).replace(tzinfo=datetime.timezone.utc).isoformat()

params = {
    "start": leyton_coords, 
    "end": harrow_on_the_hill_coords,
    "time": next_tues,
    "time_type": "depart"
    }

cm_response = requests.get(
    url=base_cm_url + "1/directions/transit",
    params=params,
    headers=headers
)

In [6]:
cm_response.json()["routes"][0].keys()

dict_keys(['start', 'end', 'distance_meters', 'duration_seconds', 'duration_accuracy', 'price', 'legs', 'route_departure_time', 'route_arrival_time', 'signature', 'requested_time', 'requested_time_type'])

From my testing, the first returned route seems to always be the quickest.

In [7]:
[data["duration_seconds"]/60 for data in cm_response.json()["routes"]]

[57.81666666666667,
 60.81666666666667,
 72.81666666666666,
 69.81666666666666,
 69.81666666666666]

## Test data

For this analysis, I need the coordinates of all London stations. I'll download the dataset based from [here](https://www.whatdotheyknow.com/request/coordinates_of_london_undergroun), which looks to be an official response to a FOI request.

In [8]:
%%script false --no-raise-error

curl -o data/raw/london_stations_coords.csv \
    https://www.whatdotheyknow.com/request/512947/response/1238210/attach/3/Stations%2020180921.csv.txt

In [9]:
london_stations_coords = pd.read_csv("../data/raw/london_stations_coords.csv")
london_stations_coords.head()

,FID,OBJECTID,NAME,EASTING,NORTHING,LINES,NETWORK,Zone,x,y
0,0,78,Temple,530959,180803,"District, Circle",London Underground,1,-0.112644,51.510474
1,1,79,Blackfriars,531694,180893,"District, Circle",London Underground,1,-0.102020,51.511114
2,2,80,Mansion House,532354,180932,"District, Circle",London Underground,1,-0.092495,51.511306
3,3,81,Cannon Street,532611,180900,"District, Circle",London Underground,1,-0.088801,51.510963
4,4,82,Monument,532912,180824,"District, Circle",London Underground,1,-0.084502,51.510209


In [10]:
london_stations_coords.shape[0]

479